In [ ]:
import pandas as pd
import glob
import re
from datetime import timedelta, datetime
import openpyxl

In [ ]:
# Finding all .xlsx files
all_files = glob.glob("*.xlsx")
print(f"Found {len(all_files)} Excel files.")

consumer_dfs = []
filename_pattern = re.compile(r'(Consumer \d+)')

Found 5 Excel files.


In [ ]:
for file_path in all_files:
    
    match = filename_pattern.search(file_path)
    if not match:
        print(f"Warning: Skipping file with unexpected name: {file_path}")
        continue
    consumer_name = match.group(1)
    
    # Reading all sheets from the Excel file
    try:
        xls_sheets = pd.read_excel(file_path, sheet_name=None)
    except Exception as e:
        print(f"Error reading {file_path}: {e}. Skipping.")
        continue

    # Looping through each sheet (e.g., 'June', 'July') in the Excel file
    for sheet_name, df in xls_sheets.items():
        print(f"Processing {file_path} -> Sheet: {sheet_name}")
        
        # Fixing "Unnamed" Columns
        columns_to_rename = {}
        first_valid_date_col = None
        for col in df.columns:
            if isinstance(col, (datetime, pd.Timestamp)):
                 # If the col is *already* a datetime, convert to string
                 first_valid_date_col = col.strftime('%Y-%m-%d')
                 columns_to_rename[col] = first_valid_date_col
            elif isinstance(col, str) and col.startswith('20'):
                 first_valid_date_col = col
                 break
        
        # If no date col found, try a different approach
        if not first_valid_date_col:
            for col in df.columns:
                 if isinstance(col, (int, float)):
                     try:
                         new_date = pd.to_datetime('1899-12-30') + pd.to_timedelta(col, 'D')
                         new_date_str = new_date.strftime('%Y-%m-%d')
                         columns_to_rename[col] = new_date_str
                         if not first_valid_date_col:
                             first_valid_date_col = new_date_str
                     except:
                         pass 

        if not first_valid_date_col:
            print(f"Warning: No valid date columns found in {file_path} -> {sheet_name}. Skipping.")
            continue

        # Now fixing Unnamed columns
        for i, col_name in enumerate(df.columns):
            if isinstance(col_name, str) and col_name.startswith('Unnamed:'):
                if i > 0:
                    prev_col_name = df.columns[i-1]
                    if prev_col_name in columns_to_rename:
                        prev_col_name = columns_to_rename[prev_col_name]
                    
                    try:
                        prev_date = pd.to_datetime(prev_col_name)
                        new_date = prev_date + timedelta(days=1)
                        columns_to_rename[col_name] = new_date.strftime('%Y-%m-%d')
                    except:
                        columns_to_rename[col_name] = f'BAD_COLUMN_{i}'
            
        df.rename(columns=columns_to_rename, inplace=True)

        #  Melt from Wide to Long format
        df.dropna(subset=['Hour'], inplace=True)
        date_columns = [col for col in df.columns if isinstance(col, str) and col.startswith('20')]

        df_melted = df.melt(id_vars=['Hour'],
                            value_vars=date_columns,
                            var_name='Date',
                            value_name='Consumption')
        
        df_melted['Consumer'] = consumer_name
        consumer_dfs.append(df_melted)

Processing Consumer 1.xlsx -> Sheet: June
Processing Consumer 1.xlsx -> Sheet: July
Processing Consumer 1.xlsx -> Sheet: August
Processing Consumer 1.xlsx -> Sheet: September
Processing Consumer 2.xlsx -> Sheet: June
Processing Consumer 2.xlsx -> Sheet: July
Processing Consumer 2.xlsx -> Sheet: August
Processing Consumer 2.xlsx -> Sheet: September
Processing Consumer 3.xlsx -> Sheet: June
Processing Consumer 3.xlsx -> Sheet: July
Processing Consumer 3.xlsx -> Sheet: August
Processing Consumer 3.xlsx -> Sheet: September
Processing Consumer 4.xlsx -> Sheet: June
Processing Consumer 4.xlsx -> Sheet: July
Processing Consumer 4.xlsx -> Sheet: August
Processing Consumer 4.xlsx -> Sheet: September
Processing Consumer 5.xlsx -> Sheet: June
Processing Consumer 5.xlsx -> Sheet: July
Processing Consumer 5.xlsx -> Sheet: August
Processing Consumer 5.xlsx -> Sheet: September


In [66]:
# Combine, Timestamp, Clean, and Save

print("\nCombining all processed files...")
all_data = pd.concat(consumer_dfs, ignore_index=True)

all_data['Hour'] = all_data['Hour'].astype(str).str.strip()
hour_is_24 = (all_data['Hour'] == '24:00')
all_data.loc[hour_is_24, 'Hour'] = '00:00'

all_data['Timestamp'] = pd.to_datetime(
    all_data['Date'] + ' ' + all_data['Hour'],
    errors='coerce'
)
all_data.dropna(subset=['Timestamp'], inplace=True)
all_data.loc[hour_is_24, 'Timestamp'] = all_data.loc[hour_is_24, 'Timestamp'] + timedelta(days=1)

all_data.set_index('Timestamp', inplace=True)
all_data.sort_index(inplace=True)

all_data['Consumption'] = all_data.groupby('Consumer')['Consumption'].transform(
    lambda x: x.interpolate(method='time')
)
all_data['Consumption'] = all_data.groupby('Consumer')['Consumption'].transform(
    lambda x: x.ffill().bfill()
)

all_data.drop(['Date', 'Hour'], axis=1, inplace=True)
all_data = all_data.reset_index().groupby(['Timestamp', 'Consumer']).mean()
all_data.to_csv('cleaned_consumption_dataset.csv')

print("\n--- Process Complete! ---")
print("Saved 'cleaned_consumption_dataset.csv'")
print(all_data.info())


Combining all processed files...

--- Process Complete! ---
Saved 'cleaned_consumption_dataset.csv'
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13178 entries, (Timestamp('2025-06-01 00:00:00'), 'Consumer 1') to (Timestamp('2025-09-30 23:00:00'), 'Consumer 5')
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Consumption  13178 non-null  float64
dtypes: float64(1)
memory usage: 164.6+ KB
None


In [67]:
all_data.head()

Consumption
Timestamp  Consumer               
2025-06-01 Consumer 1       221.67
           Consumer 2         1.22
           Consumer 3        25.70
           Consumer 4        80.34
           Consumer 5       135.91

In [68]:
all_data.describe()

,Consumption
count,13178.000000
mean,97.746236
std,76.797428
min,0.000000
25%,22.472500
50%,104.550000
75%,161.290000
max,296.660000
